In [1]:
import os
ROOT = os.path.join('..', '..')
import sys
sys.path.append(ROOT)
# 
import numpy as np

from src.models import SVDCollaborativeFiltering

In [12]:
from src.utils import read_ml, ml_train_test_split

ml_complete_df, ml_users_df, ml_ratings_df, ml_movies_df, ml_genres_df = read_ml()

MAX_K = 20
ml_ratings_train_df, ml_ratings_test_df = ml_train_test_split(ml_ratings_df=ml_ratings_df, min_user_test_samples=MAX_K)

In [13]:
recommender = SVDCollaborativeFiltering(ml_movies_df, ml_users_df)
recommender.fit(ml_ratings_train_df, n_features=10)

c:\Users\UADulinaTe\Desktop\UCU_5K2S_Recommender_Systems_Project\experiments\7_factorization_svd\..\..\src\models\factorization_svd.py:27: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  utility_matrix_df = utility_matrix_df.fillna(global_mean)


In [14]:
metrics = recommender.evaluate(ml_ratings_test_df, k=MAX_K)

100%|██████████| 769/769 [00:08<00:00, 87.69it/s]


In [15]:
print("Predictive metrics")
print("Average Precision@K:", round(np.mean(metrics["precision@k"]), 4))
print("Average Recall@K:", round(np.mean(metrics["recall@k"]), 4))
print("Average F1@K:", round(np.mean(metrics["f1@k"]), 4))
print("Average precision:", round(np.mean(metrics["average_precision"]), 4))
print("Average num of relevant items in test:", round(np.mean(metrics["relevant items"]), 4))
print()
print("Best Precision@K:", round(np.max(metrics["precision@k"]), 4))
print("Best Recall@K:", round(np.max(metrics["recall@k"]), 4))
print("Best F1@K:", round(np.max(metrics["f1@k"]), 4))
print("Best average precision", round(np.max(metrics["average_precision"]), 4))
print()
print("Ranking metrics")
print("MRR:", round(np.mean(metrics["reciprocal_rank"]), 4))
print("Average NDCG:", round(np.mean(metrics["NDCG"]), 4))

Predictive metrics
Average Precision@K: 0.346
Average Recall@K: 0.0706
Average F1@K: 0.1046
Average precision: 0.3907
Average num of relevant items in test: 140.8283

Best Precision@K: 1.0
Best Recall@K: 0.36
Best F1@K: 0.4
Best average precision 1.0

Ranking metrics
MRR: 0.6665
Average NDCG: 0.6692


In [17]:
np.max(metrics["recall@k"])


0.36